In [2]:
import pandas as pd

In [3]:
class treeNode:
    
    def __init__(self, value, numOccur, parent):
        self.name = value
        self.count = numOccur
        self.parentNode = parent
        self.link = None
        self.children = {}
        
    def inc(self, numOccur):
        self.count += numOccur
    
    def display(self, index = 1):
        print(" "*index, self.name, " : ", self.count)
        for child in self.children.values():
            child.display(index+1)

rootNode = treeNode('pyramid',9,None)
rootNode.children['eye'] = treeNode('eye',13,None)
rootNode.children['ehehe'] = treeNode('hehe',5,None)
rootNode.children['eN']  = treeNode('sdfgb',11,None)

rootNode.display()

In [4]:
def updateHeader(headerItem, itemAdding):
    
    while(headerItem.link != None):
        headerItem = headerItem.link
    headerItem.link = itemAdding

In [5]:
def updateTree(orderedItems, tree, headerTable, count):
    
    if orderedItems[0] in tree.children:
        tree.children[orderedItems[0]].inc(count)
        
    else:
        tree.children[orderedItems[0]] = treeNode(orderedItems[0], count, tree)
        
        if headerTable[orderedItems[0]][1] == None:
            headerTable[orderedItems[0]][1] = tree.children[orderedItems[0]]
        else:
            updateHeader(headerTable[orderedItems[0]][1], tree.children[orderedItems[0]])
    
    if len(orderedItems) >1:
        updateTree(orderedItems[1::], tree.children[orderedItems[0]], headerTable, count)
        

In [6]:
def createFPTree(dictdata, minSupport):
    
    headerTable = {}
    for transaction in dictdata:
        for item in transaction:
            headerTable[item] = headerTable.get( item, 0 ) + dictdata[transaction]
    #print(headerTable)
            
    #removing items with value less than minSupport
    for item in list(headerTable):
        if headerTable[item]<minSupport:
            del(headerTable[item])
            
    freqItemset = set(headerTable.keys())
    #print('freqItemset:', freqItemset)
    
    if len(freqItemset) == 0:
        return None
    
    for item in headerTable:
        headerTable[item] = [headerTable[item], None]
        #print(headerTable[item])
    
    tree = treeNode("Nullset", 1, None)
    
    for transaction, count in dictdata.items():
        localdict = {}
        for item in transaction:
            if item in freqItemset:
                localdict[item] = headerTable[item][0]
        #print("localdict \n", localdict)
        if len(localdict) > 0:
            orderedItems = []
            for v in sorted(localdict.items(), key =lambda p:p[1], reverse = True):
                orderedItems.append(v[0])
            updateTree(orderedItems, tree, headerTable, count)
        #print("orderedItems \n", orderedItems)
    return tree, headerTable, freqItemset

# 
all of the values in dicdata are initialized to 1
dictionary.get(key, integer)
if key is found:
returns the value of key in dictionary
else
returns integer

In [7]:
def getdataset(filename):
    
    df = pd.read_csv(filename)
    dataset = []
    
    for index, rows in df.iterrows():
        row = []
        for item in rows:
            row.append(item)
        dataset.append(row)
        
    return dataset

In [8]:
def createFrozenset(dataset):
    
    dictionary = {}
    for item in dataset:
        dictionary[frozenset(item)] = 1
        
    return dictionary

In [9]:
filepath = "/home/komali_priya/3-1/DWDM/dataset.csv"
df = getdataset(filepath)
#print(df)

In [10]:
dictdata = createFrozenset(df)
#print(dictdata)

In [11]:
#for  transactions in dictdata:
 #   print(dictdata[transactions])
#print(dictdata.items())

In [12]:
fpTree, headerTable, freqItemset = createFPTree(dictdata, 30)

In [13]:
fpTree.display()

  Nullset  :  1
   1  :  99
    23  :  99
     36  :  99
      67  :  99
       71  :  99
        79  :  99
         88  :  99
          90  :  99
           94  :  99
            97  :  99
             104  :  99
              128  :  95
               39  :  95
                42  :  95
                 56  :  95
                  57  :  95
                   120  :  95
                    12  :  47
                     21  :  23
                      25  :  11
                       108  :  3
                        5  :  3
                         53  :  1
                         50  :  1
                         44  :  1
                       5  :  2
                        113  :  2
                         50  :  1
                         44  :  1
                       6  :  6
                        113  :  3
                         53  :  1
                         50  :  1
                         44  :  1
                        108  :  3
                         53  : 

# Mining

In [14]:
def ascendTree(prefixPath, leafNode):
    if leafNode.parentNode != None:
        prefixPath.append(leafNode.name)
        ascendTree(prefixPath, leafNode.parentNode)

In [15]:
def findPrefixPath(item, treeNode):
    conditionalPatterns = {}
    
    while(treeNode!=None):
        prefixPath = []
        ascendTree(prefixPath, treeNode)
        
        if len(prefixPath) >1:
            conditionalPatterns[frozenset(prefixPath)] = treeNode.count
            
        treeNode = treeNode.link
    
    return conditionalPatterns
        

In [16]:
#print(headerTable[21][1])

In [17]:
#findPrefixPath(1, headerTable[1][1])

In [18]:
for item in freqItemset:
    x = findPrefixPath(item, headerTable[item][1])
    print(x)

{frozenset({128, 97, 1, 67, 36, 71, 104, 79, 23, 88, 90, 94}): 95}
{}
{frozenset({128, 1, 5, 12, 21, 23, 25, 36, 39, 42, 56, 57, 67, 71, 79, 88, 90, 94, 97, 104, 108, 120}): 3, frozenset({128, 1, 5, 12, 21, 23, 25, 36, 39, 42, 56, 57, 67, 71, 79, 88, 90, 94, 97, 104, 120}): 2, frozenset({128, 1, 5, 12, 21, 23, 26, 36, 39, 42, 56, 57, 67, 71, 79, 88, 90, 94, 97, 104, 120}): 3, frozenset({128, 1, 5, 12, 21, 23, 26, 36, 39, 42, 56, 57, 67, 71, 79, 88, 90, 94, 97, 104, 108, 120}): 3, frozenset({128, 1, 5, 12, 22, 23, 25, 36, 39, 42, 56, 57, 67, 71, 79, 88, 90, 94, 97, 104, 120}): 3, frozenset({128, 1, 5, 12, 22, 23, 25, 36, 39, 42, 56, 57, 67, 71, 79, 88, 90, 94, 97, 104, 108, 120}): 3, frozenset({128, 1, 5, 12, 22, 23, 26, 36, 39, 42, 56, 57, 67, 71, 79, 88, 90, 94, 97, 104, 120}): 3, frozenset({128, 1, 5, 12, 22, 23, 26, 36, 39, 42, 56, 57, 67, 71, 79, 88, 90, 94, 97, 104, 108, 120}): 3, frozenset({128, 1, 5, 9, 21, 23, 25, 36, 39, 42, 56, 57, 67, 71, 79, 88, 90, 94, 97, 104, 120}): 3, f